In [ ]:
"""
Solves Poisson's and Laplace's equations using the Finite Element Method (FEniCSx).

This script demonstrates how to solve two PDE problems:
1. Poisson's equation on an L-shaped domain.
2. Laplace's equation on an annular domain.

This script is a standalone Python version of the corresponding Jupyter Notebook.

Required Libraries:
- fenics-dolfinx
- mpi4py
- petsc4py
- ufl
- gmsh
- pyvista
- matplotlib
- numpy

You can typically install these with pip:
pip install fenics-dolfinx[gmsh] mpi4py petsc4py ufl pyvista matplotlib numpy
"""

!pip install fenics-dolfinx[gmsh] mpi4py petsc4py ufl pyvista matplotlib numpy
import numpy as np
import matplotlib.pyplot as plt
import os

try:
    from dolfinx import fem, mesh, plot, io
    from mpi4py import MPI
    from petsc4py.PETSc import ScalarType
    import ufl
    import gmsh
    from dolfinx.io import gmshio
    import pyvista
    print("All required libraries imported successfully.")
except ImportError as e:
    print(f"Error importing a required library: {e}")
    print("Please ensure all dependencies are installed correctly.")
    exit()

def create_l_shape_mesh(mpi_comm, refinement=0.1):
    """Creates an L-shaped mesh using gmsh."""
    gmsh.initialize()
    if mpi_comm.Get_rank() == 0:
        gmsh.model.add("l_shape")

        # Points for the L-shape
        p1 = gmsh.model.geo.addPoint(-1, -1, 0, meshSize=refinement)
        p2 = gmsh.model.geo.addPoint(1, -1, 0, meshSize=refinement)
        p3 = gmsh.model.geo.addPoint(1, 0, 0, meshSize=refinement)
        p4 = gmsh.model.geo.addPoint(0, 0, 0, meshSize=refinement) # Re-entrant corner
        p5 = gmsh.model.geo.addPoint(0, 1, 0, meshSize=refinement)
        p6 = gmsh.model.geo.addPoint(-1, 1, 0, meshSize=refinement)

        # Lines
        l1 = gmsh.model.geo.addLine(p1, p2)
        l2 = gmsh.model.geo.addLine(p2, p3)
        l3 = gmsh.model.geo.addLine(p3, p4)
        l4 = gmsh.model.geo.addLine(p4, p5)
        l5 = gmsh.model.geo.addLine(p5, p6)
        l6 = gmsh.model.geo.addLine(p6, p1)

        # Curve loop and surface
        loop = gmsh.model.geo.addCurveLoop([l1, l2, l3, l4, l5, l6])
        surface = gmsh.model.geo.addPlaneSurface([loop])

        gmsh.model.geo.synchronize()
        gmsh.model.mesh.generate(2) # Generate 2D mesh

    # Convert gmsh to dolfinx mesh
    domain, _, _ = gmshio.model_to_mesh(gmsh.model, mpi_comm, 0, 2)
    gmsh.finalize()
    return domain

def solve_l_shape_problem(mpi_comm):
    """Sets up and solves the Poisson problem on the L-shaped domain."""
    print("\n--- Solving Problem 1: Poisson Equation on L-shaped Domain ---")

    # 1. Create Mesh
    domain = create_l_shape_mesh(mpi_comm, refinement=0.05)

    # 2. Define Function Space
    V = fem.FunctionSpace(domain, ("Lagrange", 1)) # P1 elements

    # 3. Define Analytical Solution and Forcing Function
    u_D_func = lambda x: np.sin(np.pi * x[0]) * np.cos(np.pi * x[1] / 2.0)
    f_func = lambda x: -(5.0 / 4.0) * np.pi**2 * np.sin(np.pi * x[0]) * np.cos(np.pi * x[1] / 2.0)

    u_D = fem.Function(V)
    u_D.interpolate(u_D_func)
    f = fem.Function(V)
    f.interpolate(f_func)

    # 4. Define Boundary Conditions
    boundary_facets = mesh.exterior_facet_indices(domain.topology)
    boundary_dofs = fem.locate_dofs_topological(V, domain.topology.dim - 1, boundary_facets)
    bc = fem.dirichletbc(u_D, boundary_dofs)

    # 5. Define Variational Problem
    u_trial = ufl.TrialFunction(V)
    v_test = ufl.TestFunction(V)
    a = ufl.dot(ufl.grad(u_trial), ufl.grad(v_test)) * ufl.dx
    L = f * v_test * ufl.dx

    # 6. Solve the linear system
    problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    uh = problem.solve()
    uh.name = "u_h"
    print("Solved Poisson equation for L-shaped domain.")

    # 7. Post-processing and Visualization
    if mpi_comm.Get_rank() == 0:
        try:
            u_plotter = pyvista.Plotter(shape=(1, 3), window_size=[1800, 600], off_screen=True)
            topology, cells, geometry = plot.create_vtk_mesh(V)

            # Subplot 1: FEM Solution
            u_plotter.subplot(0, 0)
            grid_sol = pyvista.UnstructuredGrid(topology, cells, geometry)
            grid_sol.point_data[uh.name] = uh.x.array.real
            u_plotter.add_mesh(grid_sol, show_edges=False, cmap='viridis')
            u_plotter.add_text("FEM Solution $u_h$", position='upper_edge', font_size=10)
            u_plotter.view_xy()

            # Subplot 2: Analytical Solution
            u_plotter.subplot(0, 1)
            u_analytical_on_mesh = fem.Function(V)
            u_analytical_on_mesh.interpolate(u_D_func)
            grid_analytical = pyvista.UnstructuredGrid(topology, cells, geometry)
            grid_analytical.point_data["Analytical u_D"] = u_analytical_on_mesh.x.array.real
            u_plotter.add_mesh(grid_analytical, show_edges=False, cmap='viridis')
            u_plotter.add_text("Analytical Solution $u_D$", position='upper_edge', font_size=10)
            u_plotter.view_xy()

            # Subplot 3: Error
            u_plotter.subplot(0, 2)
            error_viz = np.abs(u_analytical_on_mesh.x.array - uh.x.array)
            grid_error = pyvista.UnstructuredGrid(topology, cells, geometry)
            grid_error.point_data["Absolute Error"] = error_viz
            u_plotter.add_mesh(grid_error, show_edges=False, cmap='Reds')
            u_plotter.add_text("Absolute Error", position='upper_edge', font_size=10)
            u_plotter.view_xy()

            u_plotter.link_views()
            u_plotter.screenshot("l_shape_solution_comparison.png")
            print("L-shape solution comparison plot saved to l_shape_solution_comparison.png")

        except Exception as e:
            print(f"An error occurred during PyVista plotting for L-shape: {e}. Skipping plot.")

    # 8. Calculate L2 error
    L2_error_form = fem.form(ufl.inner(uh - u_D, uh - u_D) * ufl.dx)
    error_local = fem.assemble_scalar(L2_error_form)
    error_global = mpi_comm.allreduce(error_local, op=MPI.SUM)

    norm_u_D_form = fem.form(ufl.inner(u_D, u_D) * ufl.dx)
    norm_u_D_local = fem.assemble_scalar(norm_u_D_form)
    norm_u_D_global = mpi_comm.allreduce(norm_u_D_local, op=MPI.SUM)

    if norm_u_D_global > 1e-12:
        relative_L2_error = np.sqrt(error_global / norm_u_D_global)
        print(f"Relative L2 error for L-shape: {relative_L2_error:.2e}")

def create_annulus_mesh(mpi_comm, r_inner=1.0, r_outer=3.0, refinement=0.1):
    """Creates an annulus mesh using gmsh."""
    gmsh.initialize()
    if mpi_comm.Get_rank() == 0:
        gmsh.model.add("annulus")
        outer_circle = gmsh.model.geo.addCircle(0, 0, 0, r_outer)
        outer_loop = gmsh.model.geo.addCurveLoop([outer_circle])
        inner_circle = gmsh.model.geo.addCircle(0, 0, 0, r_inner)
        inner_loop = gmsh.model.geo.addCurveLoop([inner_circle])
        surface = gmsh.model.geo.addPlaneSurface([outer_loop, inner_loop])
        gmsh.model.geo.synchronize()
        gmsh.option.setNumber("Mesh.MeshSizeFactor", refinement)
        gmsh.model.mesh.generate(2)

    domain, _, _ = gmshio.model_to_mesh(gmsh.model, mpi_comm, 0, 2)
    gmsh.finalize()
    return domain

def solve_annulus_problem(mpi_comm):
    """Sets up and solves the Laplace problem on the annulus domain."""
    print("\n--- Solving Problem 2: Laplace Equation on Annulus Domain ---")

    # 1. Create Mesh
    domain = create_annulus_mesh(mpi_comm, r_inner=1.0, r_outer=3.0, refinement=0.3)

    # 2. Define Function Space
    V = fem.FunctionSpace(domain, ("Lagrange", 2)) # P2 elements

    # 3. Define Analytical Solution
    def u_D_annulus_cartesian(x):
        r = np.sqrt(x[0]**2 + x[1]**2)
        r_safe = np.where(r > 1e-9, r, 1e-9)
        sin_theta = x[1] / r_safe
        return (9.0 / (8.0 * r) - r / 8.0) * sin_theta

    u_D = fem.Function(V)
    u_D.interpolate(u_D_annulus_cartesian)
    u_D.name = "u_D_annulus"

    # 4. Define Boundary Conditions
    inner_boundary = lambda x: np.isclose(np.sqrt(x[0]**2 + x[1]**2), 1.0)
    outer_boundary = lambda x: np.isclose(np.sqrt(x[0]**2 + x[1]**2), 3.0)

    u_inner_bc_func = lambda x: x[1] # y-coordinate for sin(theta) on r=1
    u_outer_bc_val = ScalarType(0.0)

    inner_facets = mesh.locate_entities_boundary(domain, domain.topology.dim - 1, inner_boundary)
    inner_dofs = fem.locate_dofs_topological(V, domain.topology.dim - 1, inner_facets)
    u_inner_expr = fem.Function(V)
    u_inner_expr.interpolate(u_inner_bc_func)
    bc_inner = fem.dirichletbc(u_inner_expr, inner_dofs)

    outer_facets = mesh.locate_entities_boundary(domain, domain.topology.dim - 1, outer_boundary)
    outer_dofs = fem.locate_dofs_topological(V, domain.topology.dim - 1, outer_facets)
    bc_outer = fem.dirichletbc(u_outer_bc_val, outer_dofs, V)

    bcs = [bc_inner, bc_outer]

    # 5. Define Variational Problem
    u_trial = ufl.TrialFunction(V)
    v_test = ufl.TestFunction(V)
    a = ufl.dot(ufl.grad(u_trial), ufl.grad(v_test)) * ufl.dx
    f_placeholder = fem.Constant(domain, ScalarType(0))
    L = f_placeholder * v_test * ufl.dx

    # 6. Solve the linear system
    problem = fem.petsc.LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    uh = problem.solve()
    uh.name = "u_h_annulus"
    print("Solved Laplace equation for annular domain.")

    # 7. Post-processing and Visualization
    if mpi_comm.Get_rank() == 0:
        try:
            plotter = pyvista.Plotter(shape=(1, 3), window_size=[1800, 600], off_screen=True)
            topology, cells, geometry = plot.create_vtk_mesh(V)

            # Subplot 1: FEM Solution
            plotter.subplot(0, 0)
            grid_sol = pyvista.UnstructuredGrid(topology, cells, geometry)
            grid_sol.point_data[uh.name] = uh.x.array.real
            plotter.add_mesh(grid_sol, show_edges=False, cmap='viridis')
            plotter.add_text("FEM Solution $u_h$", position='upper_edge', font_size=10)
            plotter.view_xy()

            # Subplot 2: Analytical Solution
            plotter.subplot(0, 1)
            u_analytical_mesh = fem.Function(V)
            u_analytical_mesh.interpolate(u_D_annulus_cartesian)
            grid_analytical = pyvista.UnstructuredGrid(topology, cells, geometry)
            grid_analytical.point_data[u_D.name] = u_analytical_mesh.x.array.real
            plotter.add_mesh(grid_analytical, show_edges=False, cmap='viridis')
            plotter.add_text("Analytical Solution $u_D$", position='upper_edge', font_size=10)
            plotter.view_xy()

            # Subplot 3: Error
            plotter.subplot(0, 2)
            error_viz = np.abs(u_analytical_mesh.x.array - uh.x.array)
            grid_error = pyvista.UnstructuredGrid(topology, cells, geometry)
            grid_error.point_data["Absolute Error"] = error_viz
            plotter.add_mesh(grid_error, show_edges=False, cmap='Reds')
            plotter.add_text("Absolute Error", position='upper_edge', font_size=10)
            plotter.view_xy()

            plotter.link_views()
            plotter.screenshot("annulus_solution_comparison.png")
            print("Annulus solution comparison plot saved to annulus_solution_comparison.png")

        except Exception as e:
            print(f"An error occurred during PyVista plotting for annulus: {e}. Skipping.")

    # 8. Calculate L2 error
    L2_error_form = fem.form(ufl.inner(uh - u_D, uh - u_D) * ufl.dx)
    error_local = fem.assemble_scalar(L2_error_form)
    error_global = mpi_comm.allreduce(error_local, op=MPI.SUM)

    norm_u_D_form = fem.form(ufl.inner(u_D, u_D) * ufl.dx)
    norm_u_D_local = fem.assemble_scalar(norm_u_D_form)
    norm_u_D_global = mpi_comm.allreduce(norm_u_D_local, op=MPI.SUM)

    if norm_u_D_global > 1e-12:
        relative_L2_error = np.sqrt(error_global / norm_u_D_global)
        print(f"Relative L2 error for annulus: {relative_L2_error:.2e}")

def main():
    """Main function to run the solvers."""
    comm = MPI.COMM_WORLD

    solve_l_shape_problem(comm)
    solve_annulus_problem(comm)

if __name__ == "__main__":
    main()
